# Dividend Capture Algorithm - Google Colab Testing Notebook

This notebook allows you to test the dividend capture algorithm in Google Colab.

## Setup Instructions:
1. Upload this notebook to Google Colab
2. Run all cells in order
3. Monitor progress and results

**Note:** This uses yfinance (free) for backtesting. Alpaca is optional.

## Step 1: Clone Repository and Install Dependencies

In [ ]:
# Clone the repository
!git clone https://github.com/DennisPoniros/XdividendALG.git

# Change to the repository directory
%cd /content/XdividendALG

# Install required packages
!pip install -q pandas numpy scipy yfinance matplotlib seaborn statsmodels scikit-learn

## Step 2: Verify Installation

In [ ]:
# Import and verify
import sys
import pandas as pd
import numpy as np

print("✅ Imports successful")
print(f"Python version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## Step 3: Validate Configuration

In [ ]:
# Validate configuration
from config import validate_config, print_config_summary

validation = validate_config()

if validation['errors']:
    print("❌ Configuration Errors:")
    for error in validation['errors']:
        print(f"   - {error}")
else:
    print("✅ Configuration validated")

if validation['warnings']:
    print("\n⚠️  Warnings:")
    for warning in validation['warnings']:
        print(f"   - {warning}")

print("\n" + "="*80)
print_config_summary()

## Step 4: Quick Validation Test (2 months)

In [ ]:
# Quick test with small dataset
from backtester import Backtester

print("🧪 Running quick validation test (Sep-Oct 2024)...\n")

bt = Backtester(
    start_date='2024-09-01',
    end_date='2024-10-31',
    initial_capital=50_000
)

results = bt.run_backtest(mode='full')

if results and results.get('total_trades', 0) > 0:
    print("\n" + "="*80)
    print("✅ VALIDATION PASSED")
    print("="*80)
    print(f"Total Trades: {results['total_trades']}")
    print(f"Win Rate: {results['win_rate_pct']:.1f}%")
    print(f"Sharpe Ratio: {results['sharpe_ratio']:.2f}")
    print(f"Total Return: {results['total_return_pct']:.2f}%")
else:
    print("\n⚠️  No trades generated (may be normal for short test period)")

## Step 5: Check Data Manager Cache Stats

In [ ]:
# Check cache efficiency
cache_stats = bt.dm.get_cache_stats()

print("\n📊 Data Manager Cache Statistics:")
print("=" * 60)
for key, value in cache_stats.items():
    print(f"  {key}: {value}")

if cache_stats['hit_rate_pct'] > 30:
    print("\n✅ Good cache efficiency - saving API calls")
else:
    print("\n⚠️  Low cache hit rate - mostly unique requests")

## Step 6: Full Backtest (2024 only - ~10 months)

In [ ]:
# Full 2024 backtest
print("\n" + "="*80)
print("🚀 Running Full 2024 Backtest")
print("="*80 + "\n")

bt_full = Backtester(
    start_date='2024-01-01',
    end_date='2024-10-31',
    initial_capital=100_000
)

results_full = bt_full.run_backtest(mode='full')

print("\n✅ Backtest Complete!")

## Step 7: Generate Performance Report

In [ ]:
# Generate analytics
if results_full:
    from analytics import create_performance_report
    
    # Create output directory in Colab
    import os
    os.makedirs('/content/outputs', exist_ok=True)
    
    plots, html_report = create_performance_report(
        results_full,
        output_dir='/content/outputs'
    )
    
    print(f"\n📁 Reports saved to /content/outputs/")
    print(f"   - {len(plots)} plots generated")
    print(f"   - HTML report: {html_report}")
    
    # Export trade log
    bt_full.export_results('/content/outputs/trades_2024.csv')
    print(f"   - Trade log: /content/outputs/trades_2024.csv")

## Step 8: Display Equity Curve

In [ ]:
# Display equity curve in notebook
from IPython.display import Image, display
import os

if os.path.exists('/content/outputs/equity_curve.png'):
    print("📈 Equity Curve:")
    display(Image('/content/outputs/equity_curve.png'))
else:
    print("⚠️  Equity curve not generated")

## Step 9: Display Drawdown Chart

In [ ]:
# Display drawdown
if os.path.exists('/content/outputs/drawdown.png'):
    print("📉 Drawdown Analysis:")
    display(Image('/content/outputs/drawdown.png'))
else:
    print("⚠️  Drawdown chart not generated")

## Step 10: View HTML Report (Optional)

In [ ]:
# Display HTML report inline
from IPython.display import IFrame

if os.path.exists('/content/outputs/backtest_report.html'):
    print("📊 Full Performance Report:")
    display(IFrame('/content/outputs/backtest_report.html', width=1000, height=800))
else:
    print("⚠️  HTML report not generated")

## Step 11: Analyze Trade Log

In [ ]:
# Load and analyze trades
import pandas as pd

if os.path.exists('/content/outputs/trades_2024.csv'):
    trades = pd.read_csv('/content/outputs/trades_2024.csv')
    
    print("\n" + "="*80)
    print("📋 TRADE LOG ANALYSIS")
    print("="*80)
    
    # Basic stats
    buys = trades[trades['action'] == 'BUY']
    sells = trades[trades['action'] == 'SELL']
    
    print(f"\nTotal Trades: {len(buys)} entries, {len(sells)} exits")
    print(f"Total Trading Costs: ${trades['costs'].sum():,.2f}")
    
    if 'pnl' in sells.columns:
        winning_trades = sells[sells['pnl'] > 0]
        losing_trades = sells[sells['pnl'] <= 0]
        
        print(f"\nWinning Trades: {len(winning_trades)}")
        print(f"Losing Trades: {len(losing_trades)}")
        print(f"Average Win: ${winning_trades['pnl'].mean():,.2f}" if len(winning_trades) > 0 else "Average Win: N/A")
        print(f"Average Loss: ${losing_trades['pnl'].mean():,.2f}" if len(losing_trades) > 0 else "Average Loss: N/A")
    
    # Show first few trades
    print("\nFirst 10 Trades:")
    display(trades.head(10))
else:
    print("⚠️  Trade log not found")

## Step 12: Download Results (Optional)

In [ ]:
# Zip all results for download
!cd /content && zip -r dividend_algo_results.zip outputs/

print("\n✅ Results packaged as 'dividend_algo_results.zip'")
print("\nTo download:")
print("  1. Click the Files icon (📁) in the left sidebar")
print("  2. Navigate to /content/")
print("  3. Right-click 'dividend_algo_results.zip' and select Download")

## Advanced: Parameter Sensitivity Test (Optional)

In [ ]:
# Test different entry windows
from config import entry_config

print("\n" + "="*80)
print("🔬 PARAMETER SENSITIVITY TEST")
print("="*80 + "\n")

test_windows = [[3], [4], [5], [3, 4], [4, 5]]
results_comparison = []

for window in test_windows:
    print(f"\nTesting entry window: {window}")
    entry_config.preferred_entry_days = window
    
    bt_test = Backtester(
        start_date='2024-07-01',
        end_date='2024-10-31',
        initial_capital=100_000
    )
    
    res = bt_test.run_backtest(mode='full')
    
    if res:
        results_comparison.append({
            'entry_window': str(window),
            'sharpe': res['sharpe_ratio'],
            'return_pct': res['total_return_pct'],
            'win_rate': res['win_rate_pct'],
            'trades': res['total_trades']
        })

# Display comparison
if results_comparison:
    comparison_df = pd.DataFrame(results_comparison)
    comparison_df = comparison_df.sort_values('sharpe', ascending=False)
    
    print("\n" + "="*80)
    print("📊 PARAMETER COMPARISON (Sorted by Sharpe)")
    print("="*80 + "\n")
    display(comparison_df)

## Summary & Next Steps

### What was tested:
1. ✅ Configuration validation
2. ✅ Quick validation test (2 months)
3. ✅ Full 2024 backtest
4. ✅ Performance analytics
5. ✅ Cache efficiency
6. ✅ Trade log analysis

### Next Steps:
1. **If results look good**: Extend backtest to longer period (2023-2024)
2. **Parameter tuning**: Adjust entry_config, exit_config based on results
3. **Paper trading**: Test with Alpaca paper account before going live
4. **Risk assessment**: Review max drawdown and VaR metrics

### Important Notes:
- ⚠️ This is historical backtesting only - past performance ≠ future results
- ⚠️ Transaction costs are included but may vary in real trading
- ⚠️ Always paper trade before risking real capital
- ⚠️ Market conditions change - monitor performance regularly